In [12]:
import duckdb as db
from deltalake import DeltaTable
from dotenv import load_dotenv
import os

# Carrega as variáveis de ambiente
load_dotenv()

# Configurações do S3
delta_path = os.getenv("DELTA_LAKE_S3_PATH")
key = os.getenv("AWS_ACCESS_KEY_ID")
secret = os.getenv("AWS_SECRET_ACCESS_KEY")
region = os.getenv("AWS_REGION")

# Função para ler a tabela Delta Lake do S3
def read_remote_delta_lake(path: str, key, secret):
    storage_options = {
        "AWS_ACCESS_KEY_ID": key,
        "AWS_SECRET_ACCESS_KEY": secret,
        "AWS_REGION": region
    }
    dt = DeltaTable(path, storage_options=storage_options)
    return dt.to_pyarrow_table()

# Carregar a tabela Delta Lake do S3
dt = read_remote_delta_lake(delta_path, key, secret)

# Conectar ao DuckDB
con = db.connect()
con.register("dt", dt)

# Função para executar consultas SQL e imprimir os resultados
def execute_query(query):
    result = con.execute(query).fetchall()
    for row in result:
        print(', '.join(map(str, row)))

# Exemplo de uso:
execute_query("SELECT * FROM dt LIMIT 5;")


INV-20945900021, 2019-07-30, 2623, Hy-Vee Food Store #4 / Sioux City, 2827 Hamilton Blvd, Sioux City, 51104, POINT (-96.417783 42.519886), 97, WOODBURY, 1031200, American Flavored Vodka, 380, Phillips Beverage, 41681, UV Red Cherry, 6, 1750, 10.99, 16.49, 6, 98.94, 10.5, 2.77
INV-20315700022, 2019-07-01, 2512, Hy-Vee Wine and Spirits / Iowa City, 1720 Waterfront Dr, Iowa City, 52240, POINT (-91.53046300000001 41.642764), 52, JOHNSON, 1062500, Flavored Rum, 035, BACARDI USA INC, 43050, Bacardi Dragon Berry, 12, 1000, 9.5, 14.25, 12, 171.0, 12.0, 3.17
INV-20430200053, 2019-07-05, 4361, Quik Trip #538 / NW 2nd / DSM, 4575 NW 2nd, Des Moines, 50313, POINT (-93.619906 41.643664), 77, POLK, 1031100, American Vodkas, 301, FIFTH GENERATION INC, 38176, Titos Handmade Vodka, 12, 750, 9.64, 14.46, 6, 86.76, 4.5, 1.18
INV-20566700011, 2019-07-12, 4392, KUM & GO #156 / Clive, 15600 Hickman Rd, Clive, 50325, POINT (-93.834209 41.614732), 77, POLK, 1062200, White Rum, 035, BACARDI USA INC, 43126, Bac